#### 1. Importing necessary libraries

In [55]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from ydata_profiling import ProfileReport
from sklearn.calibration import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

#### 2. Extracting raw data

In [56]:
raw_df_pest = pd.read_csv('data/pesticides.csv')
raw_df_rain = pd.read_csv('data/rainfall.csv')
raw_df_temp = pd.read_csv('data/temp.csv')
raw_df_yield = pd.read_csv('data/yield.csv')

display(raw_df_pest.head())
display(raw_df_rain.head())
display(raw_df_temp.head())
display(raw_df_yield.head())

,Domain,Area,Element,Item,Year,Unit,Value
0,Pesticides Use,Albania,Use,Pesticides (total),1990,tonnes of active ingredients,121.0
1,Pesticides Use,Albania,Use,Pesticides (total),1991,tonnes of active ingredients,121.0
2,Pesticides Use,Albania,Use,Pesticides (total),1992,tonnes of active ingredients,121.0
3,Pesticides Use,Albania,Use,Pesticides (total),1993,tonnes of active ingredients,121.0
4,Pesticides Use,Albania,Use,Pesticides (total),1994,tonnes of active ingredients,201.0


,Area,Year,average_rain_fall_mm_per_year
0,Afghanistan,1985,327
1,Afghanistan,1986,327
2,Afghanistan,1987,327
3,Afghanistan,1989,327
4,Afghanistan,1990,327


,year,country,avg_temp
0,1849,Côte D'Ivoire,25.58
1,1850,Côte D'Ivoire,25.52
2,1851,Côte D'Ivoire,25.67
3,1852,Côte D'Ivoire,NaN
4,1853,Côte D'Ivoire,NaN


,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value
0,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1961,1961,hg/ha,14000
1,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1962,1962,hg/ha,14000
2,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1963,1963,hg/ha,14260
3,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1964,1964,hg/ha,14257
4,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1965,1965,hg/ha,14400


Check for duplicates and null values

#### 3. Transform data

Column names are consistent and formatted


In [57]:
def format_columns(df):
    df.columns = df.columns.str.lower().str.strip()
    return df

raw_df_pest = format_columns(raw_df_pest)
raw_df_rain = format_columns(raw_df_rain)
raw_df_temp = format_columns(raw_df_temp)
raw_df_yield = format_columns(raw_df_yield)

Entries with duplicate, null, and invalid data are removed

In [58]:
# Upon inspection, raw_df_rain has values that are not numbers
raw_df_rain['average_rain_fall_mm_per_year'] = raw_df_rain['average_rain_fall_mm_per_year'].replace('..', np.nan)

In [59]:
def check_and_clean(df, name):
    print(f"{name} shape before cleaning:", df.shape)
    print(f"{name} duplicates:", df.duplicated().sum())
    print(f"{name} null values:") 
    print(df.isnull().sum())

    df = df.drop_duplicates()
    df = df.dropna()
    
    print(f"{name} shape after cleaning:", df.shape)
    print()
    return df

raw_df_pest = check_and_clean(raw_df_pest, "raw_df_pest")
raw_df_rain = check_and_clean(raw_df_rain, "raw_df_rain")
raw_df_temp = check_and_clean(raw_df_temp, "raw_df_temp")
raw_df_yield = check_and_clean(raw_df_yield, "raw_df_yield")

raw_df_pest shape before cleaning: (4349, 7)
raw_df_pest duplicates: 0
raw_df_pest null values:
domain     0
area       0
element    0
item       0
year       0
unit       0
value      0
dtype: int64
raw_df_pest shape after cleaning: (4349, 7)

raw_df_rain shape before cleaning: (6727, 3)
raw_df_rain duplicates: 0
raw_df_rain null values:
area                               0
year                               0
average_rain_fall_mm_per_year    780
dtype: int64
raw_df_rain shape after cleaning: (5947, 3)

raw_df_temp shape before cleaning: (71311, 3)
raw_df_temp duplicates: 6958
raw_df_temp null values:
year           0
country        0
avg_temp    2547
dtype: int64
raw_df_temp shape after cleaning: (62976, 3)

raw_df_yield shape before cleaning: (56717, 12)
raw_df_yield duplicates: 0
raw_df_yield null values:
domain code     0
domain          0
area code       0
area            0
element code    0
element         0
item code       0
item            0
year code       0
year            0

Columns have appropriate data types

In [60]:
def print_dtypes(df, name):
    print(f"\n{name} dtypes:")
    print(df.dtypes)

print_dtypes(raw_df_pest, "raw_df_pest")
print_dtypes(raw_df_rain, "raw_df_rain")
print_dtypes(raw_df_temp, "raw_df_temp")
print_dtypes(raw_df_yield, "raw_df_yield")


raw_df_pest dtypes:
domain      object
area        object
element     object
item        object
year         int64
unit        object
value      float64
dtype: object

raw_df_rain dtypes:
area                             object
year                              int64
average_rain_fall_mm_per_year    object
dtype: object

raw_df_temp dtypes:
year          int64
country      object
avg_temp    float64
dtype: object

raw_df_yield dtypes:
domain code     object
domain          object
area code        int64
area            object
element code     int64
element         object
item code        int64
item            object
year code        int64
year             int64
unit            object
value            int64
dtype: object


In [61]:
raw_df_rain['average_rain_fall_mm_per_year'] = raw_df_rain['average_rain_fall_mm_per_year'].astype('float64')

Normalization of data to organize entities

In [ ]:
# Create lookup tables
df_area = raw_df_yield[['area code', 'area']].drop_duplicates().reset_index(drop=True)
df_element = raw_df_yield[['element code', 'element']].drop_duplicates().reset_index(drop=True)
df_item = raw_df_yield[['item code', 'item']].drop_duplicates().reset_index(drop=True)

# Normalize raw_df_yield
df_yield = raw_df_yield.drop(columns=['area', 'element', 'item'])

# Normalize raw_df_pest
df_pest = raw_df_pest.copy()
df_pest = df_pest.merge(df_area, on='area', how='left')
df_pest = df_pest.merge(df_element, on='element', how='left')
df_pest = df_pest.merge(df_item, on='item', how='left')
df_pest = df_pest.drop(columns=['area', 'element', 'item'])

# Normalize raw_df_rain
df_rain = raw_df_rain.copy()
df_rain = df_rain.merge(df_area, left_on='area', right_on='area', how='left')
df_rain = df_rain.drop(columns=['area'])

# Normalize raw_df_temp
df_temp = raw_df_temp.copy()
df_temp = df_temp.merge(df_area, left_on='country', right_on='area', how='left')
df_temp = df_temp.drop(columns=['country'])

# Display normalized tables
display(df_area.head())
display(df_element.head())
display(df_item.head())
display(df_yield.head())
display(df_pest.head())
display(df_rain.head())
display(df_temp.head())

,area code,area
0,2,Afghanistan
1,3,Albania
2,4,Algeria
3,5,American Samoa
4,7,Angola


,element code,element
0,5419,Yield


,item code,item
0,56,Maize
1,116,Potatoes
2,27,"Rice, paddy"
3,15,Wheat
4,83,Sorghum


,domain code,domain,area code,element code,item code,year code,year,unit,value
0,QC,Crops,2,5419,56,1961,1961,hg/ha,14000
1,QC,Crops,2,5419,56,1962,1962,hg/ha,14000
2,QC,Crops,2,5419,56,1963,1963,hg/ha,14260
3,QC,Crops,2,5419,56,1964,1964,hg/ha,14257
4,QC,Crops,2,5419,56,1965,1965,hg/ha,14400


,domain,year,unit,value,area code,element code,item code
0,Pesticides Use,1990,tonnes of active ingredients,121.0,3.0,NaN,NaN
1,Pesticides Use,1991,tonnes of active ingredients,121.0,3.0,NaN,NaN
2,Pesticides Use,1992,tonnes of active ingredients,121.0,3.0,NaN,NaN
3,Pesticides Use,1993,tonnes of active ingredients,121.0,3.0,NaN,NaN
4,Pesticides Use,1994,tonnes of active ingredients,201.0,3.0,NaN,NaN


,year,average_rain_fall_mm_per_year,area code
0,1985,327.0,2.0
1,1986,327.0,2.0
2,1987,327.0,2.0
3,1989,327.0,2.0
4,1990,327.0,2.0


,year,avg_temp,area code,area
0,1849,25.58,NaN,NaN
1,1850,25.52,NaN,NaN
2,1851,25.67,NaN,NaN
3,1856,26.28,NaN,NaN
4,1857,25.17,NaN,NaN


#### 4. Loading transformed data into PostgreSQL

#### 5. Query and analyze data
- Use mathplot and seaborn for analysis
- Use PandasProfiling for full packaged analysis


In [ ]:
df = pd.read_csv('data.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

Data Checking

In [ ]:
measured_columns = ['hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']

Data Visualization

In [ ]:
for column in measured_columns:
    plt.figure(figsize=(10, 6))
    sns.displot(df[column], bins=30, kde=False, color='blue')
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

    plt.figure(figsize=(12, 8))
    sns.boxplot(data=df[column])
    plt.title(f'Box plot of {column}')
    plt.xlabel('Columns')
    plt.ylabel('Values')
    plt.show()

Time Series Analysis

In [ ]:
for column in measured_columns:
    df.groupby('Year')[column].mean().plot()
    plt.title(f'Average {column} Over Time')
    plt.xlabel('Year')
    plt.ylabel(f'Average {column}')
    plt.show()

Analysis of Yield to other columns

In [ ]:
df.groupby('Area')['hg/ha_yield'].mean().sort_values().plot(kind='bar', figsize=(15, 5))
plt.title('Average hg/ha_yield by Area')
plt.xlabel('Area')
plt.ylabel('Average hg/ha_yield')
plt.show()

df.groupby('Item')['hg/ha_yield'].mean().sort_values().plot(kind='bar', figsize=(15, 5))
plt.title('Average hg/ha_yield by Item')
plt.xlabel('Item')
plt.ylabel('Average hg/ha_yield')
plt.show()

Pairplot

In [ ]:
sns.pairplot(df[measured_columns])
plt.title('Pair Plot of All Numerical Columns')
plt.show()

Remove outliers

In [ ]:
# Remove outliers
# Remove outliers using IQR method
Q1 = df[measured_columns].quantile(0.25)
Q3 = df[measured_columns].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df[measured_columns] < (Q1 - 1.5 * IQR)) | (df[measured_columns] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_notebook_iframe()

#### 6. Apply machine learning

In [ ]:
# Encode categorical variables
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split


label_encoder_area = LabelEncoder()
df['Area'] = label_encoder_area.fit_transform(df['Area'])

label_encoder_item = LabelEncoder()
df['Item'] = label_encoder_item.fit_transform(df['Item'])

# Define features and target variable
X = df.drop(columns=['hg/ha_yield'])
y = df['hg/ha_yield']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

# Output coefficients
print("Linear Regression Coefficients:", linear_regression.coef_)
models.append(linear_regression)
model_names.append('Linear Regression')

In [ ]:
# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2

# Initialize models
decision_tree_regressor = DecisionTreeRegressor(random_state=42)
random_forest_regressor = RandomForestRegressor(random_state=42)
gradient_boosting_regressor = GradientBoostingRegressor(random_state=42)

# Fit the models
decision_tree_regressor.fit(X_train, y_train)
random_forest_regressor.fit(X_train, y_train)
gradient_boosting_regressor.fit(X_train, y_train)

# Evaluate the models
models = [decision_tree_regressor, random_forest_regressor, gradient_boosting_regressor]
model_names = ['Decision Tree', 'Random Forest', 'Gradient Boosting']

for model, name in zip(models, model_names):
    mse, r2 = evaluate_model(model, X_test, y_test)
    print(f'{name} - MSE: {mse:.4f}, R^2: {r2:.4f}')

In [ ]:
# Function to plot evaluation metrics
def plot_evaluation_metrics(model_name, y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    #  Predicted vs Actual values plot 
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'{model_name} - Predicted vs Actual')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)

    #  Residuals plot 
    residuals = y_test - y_pred
    plt.subplot(1, 2, 2)
    plt.scatter(y_pred, residuals, alpha=0.5)
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title(f'{model_name} - Residuals')
    plt.axhline(y=0, color='r', linestyle='--', lw=2)

    plt.show()

    # Print MSE and R² Score
    print(f'{model_name} - MSE: {mse:.4f}, R²: {r2:.4f}')

# Random Forest
rf = RandomForestRegressor(random_state=42, n_estimators=200)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
plot_evaluation_metrics('Random Forest', y_test, y_pred_rf)

# Decision Tree
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
plot_evaluation_metrics('Decision Tree', y_test, y_pred_dt)

# Gradient Boosting
gb = GradientBoostingRegressor(random_state=42, n_estimators=200)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
plot_evaluation_metrics('Gradient Boosting', y_test, y_pred_gb)